# Lab-3

In [82]:
import math
from decimal import Decimal, getcontext


# Basic number theory algorithms

In [83]:
def convert_to_binary_modify(number):

    binary_representation=[]

    while number>0:
    
        binary_representation.append(number & 1)
        number = number >> 1
    
    return binary_representation


In [84]:
def mod(number,module):

	numb_by_module=0

	if (number < 0):
	
		if (number // module != 0):
		
			numb_by_module = module - (abs(number) - module * (abs(number) // module))
		
		else:
		
			numb_by_module = module - abs(number)
		
	
	elif (number // module != 0):
	
		numb_by_module = number - module * (number // module)
	
	else:
	
		numb_by_module = number
	

	return numb_by_module



In [85]:
def gcd_extention( a, b): 
    q0=0
    r0=0
    r1=0
    r2=0
    u0=0
    u1=0
    u2=0

    if ((a == 0) and (b ==  0)): 
        u = v = 0
        return 0,u,v
    
    r2 = -a if (a < 0) else a

    r1 = -b if(b <  0) else b

    u2 = 1

    u1 = 0

    while (r1 != 0):
    
        q0 = r2 // r1
        r0 = r2 - q0 * r1
        r2 = r1
        r1 = r0
        u0 = u2 - q0 * u1
        u2 = u1
        u1 = u0
    
    u = u2

    if (a <  0):
        u = -(u)

    v =  0 if(b ==  0) else (r2 -(u) * a)//b

    return r2,u,v


In [86]:
def inverted_element(number,moduls):

	number = mod(number, moduls)

	gcd,u,v=gcd_extention(number, moduls)

	if (gcd ==1):
	
		b0 = moduls
		t=0
		q=0
		x0 = 0
		x1 = 1
		if (moduls == 1):
			return 1
		while (number > 1):
			q = number // moduls
			t = moduls
			moduls = number % moduls
			number = t
			t = x0
			x0 = x1 - q * x0
			x1 = t
		
		if (x1 < 0): x1 += b0

		return x1
	
	if (gcd > 1):
	
		return "Roots more or does`t exist!"
	


In [87]:
def horner_method(number,degree,module):

    binary_representation=convert_to_binary_modify(degree)

    result = 1

    for i in range(len(binary_representation)-1,-1,-1):
    
        result = (result * result) % module

        if (binary_representation[i] == 1):
        
            result = (result * number) % module
        
    return result


In [88]:
def chines_remainder_theorem(a,m):

    M = 1
    x = 0

    for i in range(0,len(m)):
        M *= m[i]
    
       
    for i in range(0,len(m)):

        n = M // m[i]
        u = inverted_element(n % m[i], m[i])
        
        x += n *u * a[i]
    

    x %= M

    if (x < 0):
        x += M
    
    general_module = M  

    return x,general_module


In [89]:
def nthroot (n, A, precision):
    getcontext().prec = precision
    
    n = Decimal(n)
    x_0 = A / n 
    x_1 = 1    
    while True:
        x_0, x_1 = x_1, (1 / n)*((n - 1)*x_0 + (A / (x_0 ** (n - 1))))
        if x_0 == x_1:
            return x_1

# Attack with small exponent applied on Сhines reminder theorem

In [90]:
e = 3
C1 = "0x3115f665a5c62cfaeb9f3f0d2dfcfe8cafb4f90a005e20ea48d9b41607ef7188"
N1 = "0xEEB25A696A48E3DAAB70EC4C4BEF7C5998A07E465C90BD37F331F5BAA80011F9"
C2 = "0x243f9d1059312b9daa01cae439cfdab7a4035364b04e5a993e43a68b79636b36"
N2 = "0xAB5F12B623D023289CB3CAE70F1849808CE0C31F9733AD6F4AC2A5564DA84F2B"
C3 = "0x20f6e6410982d39289cc4eacc04ea2ce8c853dece720f78e88963c5343c4659c"
N3 = "0x9CBDBC7A89BB945021E1924C12A78122C0E0A7E8647AF5EDF9C47A9F021A5305"

In [91]:
def attack_with_small_exponent(c_list,n_list):

    c_list=[int(c,base=16) for c in c_list]
    n_list=[int(n,base=16) for n in n_list]

    x,general_module = chines_remainder_theorem(c_list,n_list)
    
    return hex(int(nthroot (3, x, 100)))


In [92]:
attack_with_small_exponent([C1,C2,C3],[N1,N2,N3])

'0x1ffffffffffffffff00633b0b2351cfcaa22b6539734270284c1d497c7891'

# Attack-in-the-Middle

In [93]:
e = 65537
l=20
C = "0x1c2d97c113dd3bf4a699efb30ef01d12fc7d6d815b5b71fa7cdff26791d444484a0dddfe2ea677c2ea8c0b0490b4da8612dd57c6282d64e08fc720573b4e0350"
N = "0xA1BD21600C6EEE61B966343EFF6BA4D6C1F6F55A1C3440CF7C59DC31692C2CA4F2279F790FE6168B70B14D6F1C05DF799B47EF9B7E20F74E9022F7A8531CB2EF"

In [94]:
def attack_in_the_middle(l,cipher,exponent,module):

    cipher=int(cipher,base=16) 
    module=int(module,base=16) 

    X=[[i,horner_method(i,exponent,module)] for i in range(1,pow(2,l//2))]
    C_s=[(cipher*inverted_element(x[1],module)) % module  for x in X]

    T=0
    S=0
    T_e=0
    C_S=0

    for i in range(0,len(C_s)):

        triger=0

        for j in range(0,len(X)):

            if(C_s[i]==X[j][1]):
                T_e=X[j][1]
                C_S=C_s[i]
                triger=1
                break

        if triger==1:
            break

    for i in X:
        if(i[1]==T_e):
            T=i[0]
            break

    for i in X:
        if((cipher*inverted_element(i[1],module))%module==C_S):
            S=i[0]
            break

    return (T*S)%module
        

In [95]:
attack_in_the_middle(l,C,e,N)

620535

# Brute force

In [96]:
def brute_force(cipher,exponent,module):
    
    text=0

    cipher=int(cipher,base=16) 
    module=int(module,base=16) 

    while(True):
        
        if(horner_method(text,exponent,module)==cipher):
            break
        
        text+=1

    return text

In [97]:
brute_force(C,e,N)

620535

# Sources:

https://rosettacode.org/wiki/Nth_root#Python
